In [1]:
from pulp import LpMaximize,  LpProblem, lpSum, LpVariable, LpStatus, value
import pandas as pd
import time
import pulp

In [2]:
# Read data
fname = 'SampleItems.csv'

In [3]:
df = pd.read_csv(fname)
df

,Item Name,QtyOfMolds,SandWt,PourWt,UnitsRequired,MarginPerItem,CoolingDays
0,Main frame,5,120,146,20,5632,3
1,Upper Frame,10,53,72,48,3201,2
2,Vestas PLC,3,153,201,17,8973,4
3,SGRE Stage2,40,34,49,30,2007,1


In [4]:
def read_data(filename):
    
    df = pd.read_csv(filename)
    df.head()

    # prepare variables for optimization
    dictt = {}
    dictt['item_name'] = df['Item Name'].tolist()
    dictt['mld_qty'] = df['QtyOfMolds'].tolist()
    dictt['sand_req'] = df['SandWt'].tolist()
    dictt['metal_req'] = df['PourWt'].tolist()
    dictt['prod_req'] = df['UnitsRequired'].tolist()
    dictt['item_margin'] = df['MarginPerItem'].tolist()
    dictt['cooling_days'] = df['CoolingDays'].tolist()

    print (dictt.keys())
    return (dictt)

In [5]:
dictt = read_data('SampleItems.csv')
dictt

dict_keys(['item_name', 'mld_qty', 'sand_req', 'metal_req', 'prod_req', 'item_margin', 'cooling_days'])


{'item_name': ['Main frame', 'Upper Frame', 'Vestas PLC', 'SGRE Stage2'],
 'mld_qty': [5, 10, 3, 40],
 'sand_req': [120, 53, 153, 34],
 'metal_req': [146, 72, 201, 49],
 'prod_req': [20, 48, 17, 30],
 'item_margin': [5632, 3201, 8973, 2007],
 'cooling_days': [3, 2, 4, 1]}

In [6]:
# Define the problem and set up
def schedule(dictt, cap_optimizer=True, timeLimit=300,gapRel=0.05):
    print (dictt.keys())
    
    max_daily_sand = 1234 # kilograms
    max_daily_metal = 1760 # kilograms
    ndays = 13

    # no of item types is 
    nitems = len(dictt['item_name']) 
    
    print ('max daily sand, metal and number of items is', max_daily_sand,max_daily_metal, nitems)
    
    item_name = dictt['item_name']
    mld_qty = dictt['mld_qty']
    sand_req = dictt['sand_req']
    metal_req = dictt['metal_req']
    prod_req = dictt['prod_req']
    item_margin = dictt['item_margin']
    cooling_days = dictt['cooling_days']
    
    print (mld_qty, sand_req, metal_req, prod_req, item_margin, cooling_days)


    # production plan for d days, for k items
    x = LpVariable.dicts('item', ((d, k) for d in range(ndays) for k in range(nitems)), lowBound=0, upBound= 100, cat='Integer')
    print (x)

    # solving the problem
    problem = LpProblem("FoundryMaximize",LpMaximize)
    
    # maximize the objective function
    problem += lpSum(x[d, k]*item_margin[k] for d in range(ndays) for k in range(nitems)), "Objective: Maximize margin"
    
    #constraints
    #1. total  no of items to be produced is equal to demand for each items
    for k in range(nitems):
            problem += lpSum(x[d, k] for d in range(ndays)) <= prod_req[k], f"Constraint: Max quantity required for each item {k}"
        
    # max no of items that can  be produced per day is equal to no of molds
    for d in range(ndays):
        for k in range(nitems):
            problem += (x[d, k] <= mld_qty[k]), f"Constraint: Max molds available for each item {k} and day {d}"
        
    #daily total sand capacity is less than or equal to that available
    for d in range(ndays):
        problem += lpSum(x[d, k] * sand_req[k] for k in range(nitems)) <= max_daily_sand, f"Constraint: Daily maximum sand availability for day {d}"
    
    #daily total metal required is less than or equal to that available
    for d in range(ndays):
        problem += lpSum(x[d, k] * metal_req[k] for k in range(nitems)) <= max_daily_metal, f"Constraint: Daily maximum metal availability for day{d}"
    
    #cooling time for each mold must be considered
    for k in range(nitems):
        for d in range(ndays - cooling_days[k]+ 1):
            problem += lpSum(x[dd, k] for dd in range(d, d+cooling_days[k])) <= mld_qty[k], f"Constraint: cooling time for item {k} on day {d}"
        
    
    
    # Write the model to disk, not necessary
    problem.writeLP("FoundryProduction.lp")
    
    # Solve the optimization problem
    start_time = time.time()
    if cap_optimizer:
        solver = pulp.PULP_CBC_CMD(timeLimit=timeLimit, gapRel=gapRel)
        problem.solve(solver)
    else:
        problem.solve()
    print("Solved in %s seconds." % (time.time() - start_time))
    
    
    # The status of the solution is printed to the screen
    print("Status:", LpStatus[problem.status])

    # Each of the variables is printed with it's resolved optimum value
    for v in problem.variables():
        print(v.name, "=", v.varValue)
    
    ll = []
    schedule={}
    sr = 0
    mr = 0
    for d in range(ndays):
        for k in range(nitems):
            name = item_name[k]
            qty = x[d, k].value()
            ll.append([d, name, qty])
            #schedule[d, name] = qty
            #sr += qty * sand_req[k]
            #mr += qty * metal_req[k]
            #print ('no of items of {} produced on day {} is {}'.format(name, d, qty))

         
    #print ('Verification')
    #print ('maximum sand available per day is {} and used sand is {}'.format(max_daily_sand, sr))
    #print ('maximum metal available per day is {} and used metal is {}'.format(max_daily_metal, mr))
    tempdf = pd.DataFrame(ll)
    tempdf.columns = ['Day', 'Item', 'Quantity']
    
    schedule = tempdf.pivot(index='Day', columns='Item', values='Quantity').reset_index()
    
    schedule['daily_units'] = 0
    schedule['daily_sand'] = 0
    schedule['daily_metal'] = 0
    
    for i in range(nitems):
        c = item_name[i]
        schedule['daily_units'] += schedule[c]
        schedule['daily_sand'] += schedule[c] * sand_req[i]
        schedule['daily_metal'] += schedule[c] * metal_req[i]
        
    schedule.loc['Total']= schedule.sum()
    
    return (schedule)

In [7]:
sol = schedule(dictt)

dict_keys(['item_name', 'mld_qty', 'sand_req', 'metal_req', 'prod_req', 'item_margin', 'cooling_days'])
max daily sand, metal and number of items is 1234 1760 4
[5, 10, 3, 40] [120, 53, 153, 34] [146, 72, 201, 49] [20, 48, 17, 30] [5632, 3201, 8973, 2007] [3, 2, 4, 1]
{(0, 0): item_(0,_0), (0, 1): item_(0,_1), (0, 2): item_(0,_2), (0, 3): item_(0,_3), (1, 0): item_(1,_0), (1, 1): item_(1,_1), (1, 2): item_(1,_2), (1, 3): item_(1,_3), (2, 0): item_(2,_0), (2, 1): item_(2,_1), (2, 2): item_(2,_2), (2, 3): item_(2,_3), (3, 0): item_(3,_0), (3, 1): item_(3,_1), (3, 2): item_(3,_2), (3, 3): item_(3,_3), (4, 0): item_(4,_0), (4, 1): item_(4,_1), (4, 2): item_(4,_2), (4, 3): item_(4,_3), (5, 0): item_(5,_0), (5, 1): item_(5,_1), (5, 2): item_(5,_2), (5, 3): item_(5,_3), (6, 0): item_(6,_0), (6, 1): item_(6,_1), (6, 2): item_(6,_2), (6, 3): item_(6,_3), (7, 0): item_(7,_0), (7, 1): item_(7,_1), (7, 2): item_(7,_2), (7, 3): item_(7,_3), (8, 0): item_(8,_0), (8, 1): item_(8,_1), (8, 2): item_(8,

In [8]:
sol

Item,Day,Main frame,SGRE Stage2,Upper Frame,Vestas PLC,daily_units,daily_sand,daily_metal
0,0.0,5.0,0.0,0.0,3.0,8.0,1059.0,1333.0
1,1.0,0.0,0.0,5.0,0.0,5.0,265.0,360.0
2,2.0,0.0,30.0,4.0,0.0,34.0,1232.0,1758.0
3,3.0,5.0,0.0,5.0,0.0,10.0,865.0,1090.0
4,4.0,0.0,0.0,0.0,3.0,3.0,459.0,603.0
5,5.0,0.0,0.0,10.0,0.0,10.0,530.0,720.0
6,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,7.0,0.0,0.0,10.0,0.0,10.0,530.0,720.0
8,8.0,5.0,0.0,0.0,3.0,8.0,1059.0,1333.0
9,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
sol.to_csv('Result.csv')